# D-ETE305 Étude de cas en optimisation

## 1. Séance 1

### 1.1. Exemple

In [9]:
# Script to test Julia installation
include("example_BB.jl")

LoadError: SystemError: opening file "/Users/lucas/Documents/School/Supaero/3A/D-ETE/D-ETE305 Étude de cas en optimisation/example_BB.jl": No such file or directory

### 1.2. Imports

In [10]:
#packages
using JuMP
#use the solver you want
using HiGHS

### 1.3. EOD très simple

In [11]:
###########################
### 1. EOD tres simple
###########################
#create the optimization model
model = Model(HiGHS.Optimizer)
#------------------------------
#define the variables
#Nucleaire 1
@variable(model, Pnuc1 >= 0)
#Nucleaire 2
@variable(model, Pnuc2 >= 0)
#CCG
@variable(model, Pccg >= 0)
#Hydro
@variable(model, Phydro >= 0)
#Eolien
@variable(model, Peolien >= 0)
#------------------------------
#define the objective function
@objective(model, Min, 14*Pnuc1+16*Pnuc2+45*Pccg+48*Phydro+0*Peolien)
#------------------------------
#define the constraints
#la demande de 2200MWh doit être satisfaite
@constraint(model, eod, Pnuc1+Pnuc2+Pccg+Phydro+Peolien==2200)
#contraintes de production
@constraint(model, maxnuc1, Pnuc1 <= 900)
@constraint(model, maxnuc2, Pnuc2 <= 900)
@constraint(model, maxccg, Pccg <= 300)
@constraint(model, maxhydro, Phydro <= 300)
@constraint(model, maxeolien, Peolien <= 300)
#------------------------------
#print the model
print(model)
#------------------------------
#solve the model
optimize!(model)
#------------------------------
#Results
@show termination_status(model)
@show objective_value(model)
@show value(Pnuc1)
@show value(Pnuc2)
@show value(Pccg)
@show value(Phydro)
@show value(Peolien)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1 rows, 4 cols, 4 nonzeros
1 rows, 3 cols, 3 nonzeros
Presolve : Reductions: rows 1(-5); columns 3(-2); elements 3(-7)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.2600000000e+04 Pr: 1(1000) 0s
          1     3.1500000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  3.1500000000e+04
HiGHS run time      :          0.00
termination_status(model) = MathOptInterface.OPTIMAL
objective_value(model) = 31500.0
value(Pnuc1) = 900.0
value(Pnuc2) = 900.0
value(Pccg) = 100.0
value(Phydro) = 0.0
value(Peolien) = 300.0


300.0

### 1.4.  EOD avec contrainte de minimum de fonctionnement => introduction des variables binaires

In [12]:
# ###########################
# ### 2. EOD avec contrainte de minimum de fonctionnement => introduction des variables binaires
# ###########################
# create the optimization model
model = Model(HiGHS.Optimizer)
# ------------------------------
# define the variables
# Nucleaire 1
@variable(model, Pnuc1 >= 0)
# Nucleaire 1 on (UCnuc1=1) ou off (UCnuc1=0)
@variable(model, UCnuc1, Bin)
# Nucleaire 2
@variable(model, Pnuc2 >= 0)
@variable(model, UCnuc2, Bin)
# CCG
@variable(model, Pccg >= 0)
@variable(model, UCccg, Bin)
# Hydro
@variable(model, Phydro >= 0)
# Eolien
@variable(model, Peolien >= 0)
# ------------------------------
# define the objective function
@objective(model, Min, 14 * Pnuc1 + 16 * Pnuc2 + 45 * Pccg + 48 * Phydro + 0 * Peolien)
# ------------------------------
# define the constraints
# la demande de 2200MWh doit être satisfaite
@constraint(model, eod, Pnuc1 + Pnuc2 + Pccg + Phydro + Peolien == 2200)
# contraintes de production
# TODO: ajouter contraintes de Pmin et Pmax pour nuc1, nuc2 et CCG
@constraint(model,minnuc1,Pnuc1>=UCnuc1*300)
@constraint(model,minnuc2,Pnuc2>=UCnuc2*300)
@constraint(model,minccg,Pccg>=UCccg*150)

@constraint(model, maxnuc1, Pnuc1 <= 900*UCnuc1)
@constraint(model, maxnuc2, Pnuc2 <= 900*UCnuc2)
@constraint(model, maxccg, Pccg <= 300*UCccg)
@constraint(model, maxhydro, Phydro <= 300)
@constraint(model, maxeolien, Peolien <= 300)
# ------------------------------
# print the model
print(model)
# ------------------------------
# solve the model
optimize!(model)
# ------------------------------
# Results
@show termination_status(model)
@show objective_value(model)
@show value(Pnuc1)
@show value(Pnuc2)
@show value(Pccg)
@show value(Phydro)
@show value(Peolien)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
7 rows, 8 cols, 17 nonzeros
5 rows, 4 cols, 9 nonzeros
2 rows, 3 cols, 6 nonzeros
2 rows, 3 cols, 6 nonzeros

Solving MIP model with:
   2 rows
   3 cols (1 binary, 0 integer, 0 implied int., 2 continuous)
   6 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   31500           inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   31500           31800              0.94%        0      0      0         1     0.0s
 T       0       0         0   0.00%   31500           31799.999999       0.94%        0      0      0         1     0.0s

Solving report
  Status            Optimal
  Primal bound      31799.999999
  Du

300.0

### 1.5. EOD avec contrainte de rampe => problème d'optimisation sur plusieurs pas de temps et contraintes 

In [13]:
# ###########################
# ### 3. EOD avec contrainte de rampe => problème d'optimisation sur plusieurs pas de temps et contraintes supplémentaires
# ###########################
# create the optimization model
model = Model(HiGHS.Optimizer)
# ------------------------------
# define the variables
# Nombre de pas de temps
T = 3
# Demande pour chaque pas de temps
demande = [2200, 2450, 1900]
# Nucleaire 1
@variable(model, Pnuc1[1:T] >= 0)
# Nucleaire 1 on (UCnuc1=1) ou off (UCnuc1=0)
@variable(model, UCnuc1[1:T], Bin)
# Nucleaire 2
@variable(model, Pnuc2[1:T] >= 0)
@variable(model, UCnuc2[1:T], Bin)
# CCG
@variable(model, Pccg[1:T] >= 0)
@variable(model, UCccg[1:T], Bin)
# Hydro
@variable(model, Phydro[1:T] >= 0)
# Eolien
@variable(model, Peolien[1:T] >= 0)
# ------------------------------
# define the objective function
@objective(model, Min, sum(14 * Pnuc1[t] + 16 * Pnuc2[t] + 45 * Pccg[t] + 48 * Phydro[t] + 0 * Peolien[t] for t in 1:T))
# ------------------------------
# define the constraints
# la demande doit être satisfaite à chaque heure t
@constraint(model, eod[t in 1:T], Pnuc1[t] + Pnuc2[t] + Pccg[t] + Phydro[t] + Peolien[t] == demande[t])
# contraintes de production
# TODO: ajouter contraintes de min et max pour nuc1, nuc2 et CCG
@constraint(model,minnuc1[t in 1:T],Pnuc1[t]>=UCnuc1[t]*300)
@constraint(model,minnuc2[t in 1:T],Pnuc2[t]>=UCnuc2[t]*300)
@constraint(model,minccg[t in 1:T],Pccg[t]>=UCccg[t]*150)

@constraint(model, maxnuc1[t in 1:T], Pnuc1[t] <= 900*UCnuc1[t])
@constraint(model, maxnuc2[t in 1:T], Pnuc2[t] <= 900*UCnuc2[t])
@constraint(model, maxccg[t in 1:T], Pccg[t] <= 300*UCccg[t])

@constraint(model, maxhydro[t in 1:T], Phydro[t] <= 300)
@constraint(model, maxeolien[t in 1:T], Peolien[t] <= 300)
# contraintes de limitation de variation de la puissance
# TODO: ajouter contraintes de rampe pour nuc1, nuc2 et CCG
@constraint(model, rampenuc1[t in 2:T], Pnuc1[t] - Pnuc1[t-1] <= 350)
@constraint(model, rampenuc1_abs[t in 2:T], Pnuc1[t-1] - Pnuc1[t] <= 350)
@constraint(model, rampenuc2[t in 2:T], Pnuc2[t] - Pnuc2[t-1] <= 350)
@constraint(model, rampenuc2_abs[t in 2:T], Pnuc2[t-1] - Pnuc2[t] <= 350)
@constraint(model, rampeccg[t in 2:T], Pccg[t] - Pccg[t-1] <= 200)
@constraint(model, rampeccg_abs[t in 2:T], Pccg[t-1] - Pccg[t] <= 200)
# ------------------------------
# print the model
print(model)
# ------------------------------
# solve the model
optimize!(model)
# ------------------------------
# Results
@show termination_status(model)
@show objective_value(model)
@show value.(Pnuc1)
@show value.(Pnuc2)
@show value.(Pccg)
@show value.(Phydro)
@show value.(Peolien)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
33 rows, 24 cols, 75 nonzeros
24 rows, 21 cols, 65 nonzeros
22 rows, 14 cols, 56 nonzeros
13 rows, 13 cols, 41 nonzeros
13 rows, 13 cols, 41 nonzeros

Solving MIP model with:
   13 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   41 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   88150           inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   98500           98800              0.30%        0      0      0         5     0.0s

Solving report
  Status            Optimal
  Primal bound      98800
  Dual bound        98800
  Gap               0% (tolerance: 0.01%)
  Solution status   

3-element Vector{Float64}:
 300.0
 300.0
 300.0

### 1.6. EOD avec contrainte de Dmin => ajout de variables binaires supplémentaires

In [14]:
# ###########################
# ### 4. EOD avec contrainte de Dmin => ajout de variables binaires supplémentaires
# ###########################
# create the optimization model
model = Model(HiGHS.Optimizer)
# ------------------------------
# define the variables
# Nombre de pas de temps
T = 3
# Demande pour chaque pas de temps
demande = [2200, 2450, 1900]
# Nucleaire 1
@variable(model, Pnuc1[1:T] >= 0)
# Nucleaire 1 on (UCnuc1=1) ou off (UCnuc1=0)
@variable(model, UCnuc1[1:T], Bin)
@variable(model, UPnuc1[1:T], Bin)
@variable(model, DOnuc1[1:T], Bin)
# Nucleaire 2
@variable(model, Pnuc2[1:T] >= 0)
@variable(model, UCnuc2[1:T], Bin)
@variable(model, UPnuc2[1:T], Bin)
@variable(model, DOnuc2[1:T], Bin)
# CCG
@variable(model, Pccg[1:T] >= 0)
@variable(model, UCccg[1:T], Bin)
@variable(model, UPccg[1:T], Bin)
@variable(model, DOccg[1:T], Bin)
# Hydro
@variable(model, Phydro[1:T] >= 0)
# Eolien
@variable(model, Peolien[1:T] >= 0)
# ------------------------------
# define the objective function
@objective(model, Min, sum(14 * Pnuc1[t] + 16 * Pnuc2[t] + 45 * Pccg[t] + 48 * Phydro[t] + 0 * Peolien[t] for t in 1:T))
# ------------------------------
# define the constraints
# la demande doit être satisfaite à chaque heure t
@constraint(model, eod[t in 1:T], Pnuc1[t] + Pnuc2[t] + Pccg[t] + Phydro[t] + Peolien[t] == demande[t])
# contraintes de production
# TODO: ajouter contraintes de min et max pour nuc1, nuc2 et CCG
@constraint(model,minnuc1[t in 1:T],Pnuc1[t]>=UCnuc1[t]*300)
@constraint(model,minnuc2[t in 1:T],Pnuc2[t]>=UCnuc2[t]*300)
@constraint(model,minccg[t in 1:T],Pccg[t]>=UCccg[t]*150)

@constraint(model, maxnuc1[t in 1:T], Pnuc1[t] <= 900*UCnuc1[t])
@constraint(model, maxnuc2[t in 1:T], Pnuc2[t] <= 900*UCnuc2[t])
@constraint(model, maxccg[t in 1:T], Pccg[t] <= 300*UCccg[t])

@constraint(model, maxhydro[t in 1:T], Phydro[t] <= 300)
@constraint(model, maxeolien[t in 1:T], Peolien[t] <= 300)
# contraintes de limitation de variation de la puissance
# TODO: ajouter contraintes de rampe pour nuc1, nuc2 et CCG
@constraint(model, rampenuc1[t in 2:T], Pnuc1[t] - Pnuc1[t-1] <= 350)
@constraint(model, rampenuc1_abs[t in 2:T], Pnuc1[t-1] - Pnuc1[t] <= 350)
@constraint(model, rampenuc2[t in 2:T], Pnuc2[t] - Pnuc2[t-1] <= 350)
@constraint(model, rampenuc2_abs[t in 2:T], Pnuc2[t-1] - Pnuc2[t] <= 350)
@constraint(model, rampeccg[t in 2:T], Pccg[t] - Pccg[t-1] <= 200)
@constraint(model, rampeccg_abs[t in 2:T], Pccg[t-1] - Pccg[t] <= 200)
# contraintes liant UC, UP et DO
# TODO: ajouter contraintes liant UC, UP et DO pour nuc1, nuc2 et CCG (attention penser à l'initialisation de UP et DO)
# Il y a 4 contraintes pour chaque moyen de production donc 12 contraintes au total
@constraint(model, fctnuc1[t in 2:T], UCnuc1[t]-UCnuc1[t-1]==UPnuc1[t]-DOnuc1[t])
@constraint(model, UPDOnuc1[t in 1:T], UPnuc1[t]+DOnuc1[t]<=1)
@constraint(model, iniUPnuc1, UPnuc1[1]==UCnuc1[1])
@constraint(model, iniDOnuc1, DOnuc1[1]==1-UCnuc1[1])

@constraint(model, fctnuc2[t in 2:T], UCnuc2[t]-UCnuc2[t-1]==UPnuc2[t]-DOnuc2[t])
@constraint(model, UPDOnuc2[t in 1:T], UPnuc2[t]+DOnuc2[t]<=1)
@constraint(model, iniUPnuc2, UPnuc2[1]==UCnuc2[1])
@constraint(model, iniDOnuc2, DOnuc2[1]==1-UCnuc2[1])

@constraint(model, fctccg[t in 2:T], UCccg[t]-UCccg[t-1]==UPccg[t]-DOccg[t])
@constraint(model, UPDOccg[t in 1:T], UPccg[t]+DOccg[t]<=1)
@constraint(model, iniUPccg, UPccg[1]==UCccg[1])
@constraint(model, iniDOccg, DOccg[1]==1-UCccg[1])

# contraintes de Dmin de fonctionnement
# TODO: ajouter contraintes de dmin pour la marche pour nuc1, nuc2 et CCG
# Il y a 2 contraintes (une pour t<Tmin et une autre pour T>=Tmin) pour chaque moyen de production donc 6 contraintes au total
@constraint(model, dminnuc1[t in min(24,T):T], UCnuc1[t] >= sum(UPnuc1[i] for i in (t-min(24,T)+1):t))
@constraint(model, dminnuc1_init[t in 1:2], UCnuc1[t] >= sum(UPnuc1[i] for i in 1:t))

@constraint(model, dminnuc2[t in min(24,T):T], UCnuc2[t] >= sum(UPnuc2[i] for i in (t-min(24,T)+1):t))
@constraint(model, dminnuc2_init[t in 1:2], UCnuc2[t] >= sum(UPnuc2[i] for i in 1:t))

@constraint(model, dminccg[t in 3:T], UCccg[t] >= sum(UPccg[i] for i in (t-3+1):t))
@constraint(model, dminccg_init[t in 1:2], UCccg[t] >= sum(UPccg[i] for i in 1:t))
# ------------------------------
# print the model
print(model)
# ------------------------------
# solve the model
optimize!(model)
# ------------------------------
# Results
@show termination_status(model)
@show objective_value(model)
@show value.(Pnuc1)
@show value.(Pnuc2)
@show value.(Pccg)
@show value.(Phydro)
@show value.(Peolien)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
51 rows, 36 cols, 132 nonzeros
40 rows, 28 cols, 122 nonzeros
33 rows, 23 cols, 105 nonzeros
24 rows, 13 cols, 54 nonzeros
10 rows, 12 cols, 35 nonzeros
10 rows, 12 cols, 35 nonzeros

Solving MIP model with:
   10 rows
   12 cols (1 binary, 0 integer, 0 implied int., 11 continuous)
   35 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   92800           inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   102550          104000             1.39%        0      0      0         4     0.0s
 L       0       0         0   0.00%   103150          103150             0.00%        4      2      0         8     0.0s

So

3-element Vector{Float64}:
 300.0
 300.0
 300.0

## 2. Séance 2

In [15]:
#packages
using JuMP
#use the solver you want
using HiGHS
#package to read excel files
using XLSX

In [45]:
Tmax = 168 #optimization for 1 week (7*24=168 hours)
data_file = "Seance2/data_eod_1_week_winter.xlsx"
#data for load and fatal generation
load = XLSX.readdata(data_file, "data", "C4:C171")
wind = XLSX.readdata(data_file, "data", "D4:D171")
solar = XLSX.readdata(data_file, "data", "E4:E171")
hydro_fatal = XLSX.readdata(data_file, "data", "F4:F171")
thermal_fatal = XLSX.readdata(data_file, "data", "G4:G171")
#total of RES
Pres = wind + solar + hydro_fatal + thermal_fatal

#data for thermal clusters
Nth = 5 #number of thermal generation units
names = XLSX.readdata(data_file, "data", "J4:J8")
dict_th = Dict(i=> names[i] for i in 1:Nth)
costs_th = XLSX.readdata(data_file, "data", "K4:K8")
Pmin_th = XLSX.readdata(data_file, "data", "M4:M8") #MW
Pmax_th = XLSX.readdata(data_file, "data", "L4:L8") #MW
dmin = XLSX.readdata(data_file, "data", "N4:N8") #hours

#data for hydro reservoir
Nhy = 1 #number of hydro generation units
Pmin_hy = zeros(Nhy)
Pmax_hy = XLSX.readdata(data_file, "data", "R4") *ones(Nhy) #MW
e_hy = XLSX.readdata(data_file, "data", "S4")*ones(Nhy) #MWh
costs_hy = XLSX.readdata(data_file, "data", "Q4")*ones(Nhy) #MWh

#costs
cth = repeat(costs_th', Tmax) #cost of thermal generation €/MWh
chy = repeat(costs_hy', Tmax) #cost of hydro generation €/MWh
cuns = 5000*ones(Tmax) #cost of unsupplied energy €/MWh
cexc = 0*ones(Tmax) #cost of in excess energy €/MWh



#data for STEP/battery
#weekly STEP
Pmax_STEP = XLSX.readdata(data_file, "data", "R5") #MW
rSTEP = XLSX.readdata(data_file, "data", "T5")
max_stock_STEP = XLSX.readdata(data_file, "data", "S5") #MWh


#battery
Pmax_battery = 280 #MW
rbattery = 0.85
d_battery = 2 #hours

2

In [48]:
#############################
#create the optimization model
#############################
model = Model(HiGHS.Optimizer)

#############################
#define the variables
#############################
#thermal generation variables
@variable(model, Pth[1:Tmax,1:Nth] >= 0)
@variable(model, UCth[1:Tmax,1:Nth], Bin)
@variable(model, UPth[1:Tmax,1:Nth], Bin)
@variable(model, DOth[1:Tmax,1:Nth], Bin)
#hydro generation variables
@variable(model, Phy[1:Tmax,1:Nhy] >= 0)
#unsupplied energy variables
@variable(model, Puns[1:Tmax] >= 0)
#in excess energy variables
@variable(model, Pexc[1:Tmax] >= 0)
#weekly STEP variables
@variable(model, Pcharge_STEP[1:Tmax] >= 0)
@variable(model, Pdecharge_STEP[1:Tmax] >= 0)
@variable(model, stock_STEP[1:Tmax] >= 0)
#battery variables
@variable(model, Pcharge_battery[1:Tmax] >= 0)
@variable(model, Pdecharge_battery[1:Tmax] >= 0)
@variable(model, stock_battery[1:Tmax] >= 0)
#
# #############################
#define the objective function
#############################
@objective(model, Min, sum(Pth.*cth)+sum(Phy.*chy)+Puns'cuns+Pexc'cexc)

#############################
#define the constraints
#############################
#balance constrain
@constraint(model, balance[t in 1:Tmax], sum(Pth[t,g] for g in 1:Nth) + sum(Phy[t,h] for h in 1:Nhy) + Pres[t] + Puns[t] - load[t] - Pexc[t] - Pcharge_STEP[t] + Pdecharge_STEP[t] - Pcharge_battery[t] + Pdecharge_battery[t]== 0)
#thermal unit Pmax constraints
@constraint(model, max_th[t in 1:Tmax, g in 1:Nth], Pth[t,g] <= Pmax_th[g]*UCth[t,g])
#thermal unit Pmin constraints
@constraint(model, min_th[t in 1:Tmax, g in 1:Nth], Pmin_th[g]*UCth[t,g] <= Pth[t,g])
#thermal unit Dmin constraints
for g in 1:Nth
        if (dmin[g] > 1)
            @constraint(model, [t in 2:Tmax], UCth[t,g]-UCth[t-1,g]==UPth[t,g]-DOth[t,g],  base_name = "fct_th_$g")
            @constraint(model, [t in 1:Tmax], UPth[t]+DOth[t]<=1,  base_name = "UPDOth_$g")
            @constraint(model, UPth[1,g]==0,  base_name = "iniUPth_$g")
            @constraint(model, DOth[1,g]==0,  base_name = "iniDOth_$g")
            @constraint(model, [t in dmin[g]:Tmax], UCth[t,g] >= sum(UPth[i,g] for i in (t-dmin[g]+1):t),  base_name = "dminUPth_$g")
            @constraint(model, [t in dmin[g]:Tmax], UCth[t,g] <= 1 - sum(DOth[i,g] for i in (t-dmin[g]+1):t),  base_name = "dminDOth_$g")
            @constraint(model, [t in 1:dmin[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t), base_name = "dminUPth_$(g)_init")
            @constraint(model, [t in 1:dmin[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t), base_name = "dminDOth_$(g)_init")
    end
end

#hydro unit constraints
@constraint(model, unit_bounds_hy[t in 1:Tmax, h in 1:Nhy], Pmin_hy[h] <= Phy[t,h] <= Pmax_hy[h])
#hydro stock constraint
@constraint(model, stock_bounds_hy[h in 1:Nhy], sum(Phy[t,h] for t in 1:Tmax) <= e_hy[h])

#weekly STEP
@constraint(model, max_Pcharge_STEP[t in 1:Tmax],Pcharge_STEP[t] <= Pmax_STEP)
@constraint(model, max_Pdecharge_STEP[t in 1:Tmax],Pdecharge_STEP[t] <= Pmax_STEP)

@constraint(model, stock_STEP_max[t in 1:Tmax], stock_STEP[t] <= max_stock_STEP)
@constraint(model, stock_STEP_evo[t in 2:Tmax],stock_STEP[t] == stock_STEP[t-1] + Pcharge_STEP[t]*rSTEP - Pdecharge_STEP[t])
@constraint(model, stock_STEP_init[t in 1:Tmax],stock_STEP[1] == 0.5*max_stock_STEP)

#battery
@constraint(model, max_Pcharge_battery[t in 1:Tmax], Pcharge_battery[t] <= Pmax_battery)
@constraint(model, max_Pdecharge_battery[t in 1:Tmax], Pdecharge_battery[t] <= Pmax_battery)

@constraint(model, stock_battery_max[t in 1:Tmax], stock_battery[t] <= Pmax_battery*d_battery)
@constraint(model, stock_battery_evo[t in 2:Tmax],stock_battery[t] == stock_battery[t-1] + Pcharge_battery[t]*rbattery - Pdecharge_battery[t]/rbattery)
@constraint(model, stock_battery_init[t in 1:Tmax],stock_battery[1] == 0)



#no need to print the model when it is too big
#solve the model
optimize!(model)
#------------------------------
#Results
@show termination_status(model)
@show objective_value(model)
@show value.(stock_battery[t] for t in 1:Tmax)

#exports results as csv file
th_gen = value.(Pth)
hy_gen = value.(Phy)
STEP_charge = value.(Pcharge_STEP)
STEP_decharge = value.(Pdecharge_STEP)
battery_charge = value.(Pcharge_battery)
battery_decharge = value.(Pdecharge_battery)


# new file created
touch("results.csv")

# file handling in write mode
f = open("results.csv", "w")

for name in names
    write(f, "$name ;")
end
write(f, "Hydro ; STEP pompage ; STEP turbinage ; Batterie injection ; Batterie soutirage ; RES ; load ; Net load \n")

for t in 1:Tmax
    for g in 1:Nth
        write(f, "$(th_gen[t,g]) ; ")
    end
    for h in 1:Nhy
        write(f, "$(hy_gen[t,h]) ;")
    end
    write(f, "$(STEP_charge[t]) ; $(STEP_decharge[t]) ;")
    write(f, "$(battery_charge[t]) ; $(battery_decharge[t]) ;")
    write(f, "$(Pres[t]) ;  $(load[t]) ; $(load[t]-Pres[t]) \n")

end

close(f)


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4187 rows, 4188 cols, 22286 nonzeros
3377 rows, 3734 cols, 24945 nonzeros
3333 rows, 3690 cols, 24857 nonzeros

Solving MIP model with:
   3333 rows
   3690 cols (1560 binary, 0 integer, 0 implied int., 2130 continuous)
   24857 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -71413110       inf                  inf        0      0      0         0     0.1s
         0       0         0   0.00%   14101654.92042  inf                  inf        0      0      4       982     0.2s
 R       0       0         0   0.00%   14101654.92042  17640089.71453    20.06%      323     39      4      1227     0.2s
 L       0       0         0   0.00%   14101654.92042  14101654